In [ ]:

# CRM 분석이란 무엇인가?

# CRM 분석은 귀사의 판매 및 고객 서비스 성과를 입증하는 데이터입니다. 또한 CRM 분석은 보다 현명한 비즈니스 의사 결정을 알리는 데 사용할 수 있는 고객 데이터를 제공합니다. 일반적으로 CRM 소프트웨어를 사용하여 CRM 분석을 얻고 모든 데이터 수집 및 보고서 생성을 자동화합니다.

# CRM 분석의 이점

# CRM 분석의 주요 이점은 이를 사용하여 영업, 고객 서비스 및 마케팅 프로세스를 알릴 수 있다는 것입니다. CRM 분석을 사용하여 다음을 통해 방법을 개선할 수 있습니다.

# 고객 서비스 평가. CRM 분석은 고객 서비스 팀의 성과를 알려줍니다. 팀이 개선할 수 있는 수치를 발견하면, 이러한 목표를 향해 팀을 추진하는 관행을 구현하십시오.
# 정확한 고객 데이터. 고객 데이터를 인구 통계 마케팅 또는 전자 메일 마케팅에 사용하든, 적합한 사람에게 연락하고 있는지 알아야 합니다. CRM 분석은 당신이 그것을 하고 있다는 것을 보장한다.
# 철저한 고객 분석. 당신의 고객은 보통 한 분기당 얼마를 소비합니까? 같은 제품을 몇 번이고 사는 건가요, 아니면 다른 건가요? CRM 분석을 통해 이러한 질문에 대한 확실한 답을 얻을 수 있으며, 배운 내용을 사용하여 마케팅 전략을 개선할 수 있습니다.
# 효율적인 리드 생성. 당신의 CRM 분석은 당신의 마케팅 노력 중 어떤 것이 구매와 가장 강하게 관련이 있는지 알려줄 수 있다. 구매와 밀접한 관련이 있는 한 가지 접근 방식이 있지만 이러한 접근 방식을 통해 일부 고객만을 대상으로 했다면, 이 방법을 더 시도해 보십시오. 즉, 매출이 증가할 수 있습니다.
# 데이터 셋 설명 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 


In [56]:
import pandas as pd
import warnings
import os
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import timedelta

from lifetimes.plotting import *
from lifetimes.utils import *
from lifetimes import BetaGeoFitter
from lifetimes.fitters.gamma_gamma_fitter import GammaGammaFitter

from hyperopt import hp, fmin, tpe, rand, SparkTrials, STATUS_OK, space_eval, Trials
# gridsearch와 유사한 머신러닝 하이퍼파라미터 튜닝



In [57]:
dataset =  pd.read_csv('/Users/hyeonchanglee/Documents/data_analyst/004마케팅_퍼널/CRM_dataset/CRM_dataset.csv', \
    encoding = 'unicode_escape',dtype = {'CustomerID': str,'InvoiceID': str},)

dataset['InvoiceDate'] = pd.to_datetime(dataset['InvoiceDate'])

df = dataset.copy()
print(df)

       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

               InvoiceDate  UnitPrice CustomerID         Country  
0      2

In [ ]:
df.isnull().sum(),  df.duplicated().sum()



In [58]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.date

# print(df['InvoiceDate'])

#null
# df.isnull().sum()
df.dropna(inplace=True) #널값 제거
# print(df.isnull().sum())

#duplicates
df.drop_duplicates(inplace=True) #중복 제거
print(df.duplicated().sum())

#cancel
df = df[df['Quantity']>0] # 0 이하 제거
df = df[~df['InvoiceNo'].str.contains('C',na=False)] # C 값 제거

print(df['Quantity'].describe())
print(df[df['InvoiceNo'].str.contains('C')])


0
count    392731.000000
mean         13.153746
std         181.588650
min           1.000000
25%           2.000000
50%           6.000000
75%          12.000000
max       80995.000000
Name: Quantity, dtype: float64
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


In [59]:
df['Sales'] = df['Quantity'] * df['UnitPrice'] # Sales생성

In [60]:
cols = ['CustomerID','InvoiceNo','InvoiceDate','Sales']
df = df[cols]
print(df)

       CustomerID InvoiceNo InvoiceDate  Sales
0           17850    536365  2010-12-01  15.30
1           17850    536365  2010-12-01  20.34
2           17850    536365  2010-12-01  22.00
3           17850    536365  2010-12-01  20.34
4           17850    536365  2010-12-01  20.34
...           ...       ...         ...    ...
541904      12680    581587  2011-12-09  10.20
541905      12680    581587  2011-12-09  12.60
541906      12680    581587  2011-12-09  16.60
541907      12680    581587  2011-12-09  16.60
541908      12680    581587  2011-12-09  14.85

[392731 rows x 4 columns]


In [ ]:
df.head()

In [171]:
#RFMT를 쉽게 계산해주는 함수
# from lifetimes.utils import *
# summary_data_from_transaction_data

current_date = df['InvoiceDate'].max() #2011-12-09


metrics_df = summary_data_from_transaction_data(df,
                                    customer_id_col='CustomerID', # 고객아이디
                                    datetime_col='InvoiceDate',   # 일자
                                    monetary_value_col='Sales',   # 매출
                                    observation_period_end=current_date) # 현재날짜

metrics_df.head()



# frequency           6.000000
# recency           365.000000
# T                 367.000000
# monetary_value    599.701667
# Name: 12347, dtype: float64


# 아이디 12347은 
# 총 6번 구매했고 (frequency),
# 마지막 구매일 - 첫 구매일은 365일 (recency),
# 집계일 - 첫 구매일은 367일 (T)이며,
# 평균 구매 금액은 £600 정도 (monetary_value)임을 의미합니다.


,frequency,recency,T,monetary_value
CustomerID,,,,
12346,0.0,0.0,325.0,0.000000
12347,6.0,365.0,367.0,599.701667
12348,3.0,283.0,358.0,301.480000
12349,0.0,0.0,18.0,0.000000
12350,0.0,0.0,310.0,0.000000


In [112]:
df.groupby('CustomerID').get_group('12347')['InvoiceNo'].nunique()

data_12347 = df.groupby('CustomerID').get_group('12347')
# data_12347.groupby('InvoiceNo')['InvoiceDate'].count()

data_12347.groupby('InvoiceDate').sum()[1:].mean() # 날짜별로 그룹화 한 후 구매금액의 합을 날짜별 평균을 구함


Sales    599.701667
dtype: float64

In [ ]:
# # Customer Lifetime Value


# cltv_df = df.groupby('CustomerID').agg({
#     'InvoiceDate': [lambda x : (x.max() - x.min()).days,
#                     lambda x : (today_date - x.min()).days], # 이런식으로 리스트 안에 두가지 식을 적용 시 컬럼이 각각 만들어짐
#     'InvoiceNo': lambda x : x.nunique(),
#     'TotalPrice' : lambda x : x.sum()
# })

# # cltv_df.head()
# # InvoiceDate	InvoiceNo	TotalPrice
# # <lambda_0>	<lambda_1>	<lambda>	<lambda>

# # T 컬럼은 오늘부터 가장 처음 구매한 기간의 차로 구매 기간을 의미
# cltv_df.columns = cltv_df.columns.droplevel(0)
# cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
# cltv_df.head()

In [169]:
from tqdm import tqdm

map_money = df.groupby(['CustomerID','InvoiceDate'])['Sales'].sum()  #고객 그룹 -> 일자별 그룹 매출합.

# map_money.groupby('CustomerID').agg(lambda x : )

# for e,i in tqdm(enumerate(map_money.index)):
#     if e == 20:
#         break

#     map_money = df.groupby(['CustomerID','InvoiceDate'])['Sales'].sum()
#     cos_x = map_money.groupby('CustomerID').get_group(i[0])
#     # print(cos_x)
#     cos_x.iloc[0] = 0
#     # print(cos_x)

# print(map_money)



# for i in tqdm(map_money.index):
#     map_money = df.groupby(['CustomerID','InvoiceDate'])['Sales'].sum()
#     cos_x = map_money.groupby('CustomerID').get_group(i[0])
#     # print(cos_x)
#     cos_x.iloc[0] = 0
#     # print(cos_x)

# print(map_money)


# map_money.groupby('InvoiceDate').sum()

CustomerID  InvoiceDate
12346       2011-01-18     77183.60
12347       2010-12-07       711.79
            2011-01-26       475.39
            2011-04-07       636.25
            2011-06-09       382.52
                             ...   
18283       2011-11-30       220.31
            2011-12-06       208.00
18287       2011-05-22       765.28
            2011-10-12      1001.32
            2011-10-28        70.68
Name: Sales, Length: 16766, dtype: float64

In [ ]:
# penalizer_coef


# model = BetaGeoFitter(penalizer_coef=l2_reg)
# model = GammaGammaFitter(penalizer_coef=l2_reg)

## 훈련 /테스트 데이터 나누기

In [174]:

# calibration_and_holdout_data  함수
# calibration 을 훈련 데이터라고 함
# holdout 을 테스트 데이터라고 함 (2:1 혹은 마지막 3달을 둠)

holdout_days = 90
calibration_end_date = current_date - timedelta(days = holdout_days) 
current_date = df['InvoiceDate'].max() #2011-12-09

# 현재 날짜에서 90일을 뺀 날짜를 계산해줌

# print(calibration_end_date)

metrics_cal_df = calibration_and_holdout_data(df,
                                            customer_id_col='CustomerID',
                                            datetime_col='InvoiceDate',
                                            calibration_period_end=calibration_end_date,
                                            observation_period_end=current_date,
                                            monetary_value_col='Sales')


# customer_id_col='Customer' # 고객 아이디가 있는 컬럼명
# datetime_col='InvoiceDate', # 일자가 있는 컬럼명
# calibration_period_end=calibration_end_date, # 훈련 데이터가 끝난는 날짜
# observation_period_end=current_date, # 데이터가 끝나는 날짜
# monetary_value_col='Sales') # 금액이 있는 컬럼명
metrics_cal_df


# calibration 기간 동안의 RFMT를 계산한 값은 _cal이 붙고, 
# holdout 기간 동안의 F와 M을 계산한 값은 _holdout이 붙습니다.
# duration_holdout은 holdout data이 며칠인지 나타내는 컬럼입니다.

,frequency_cal,recency_cal,T_cal,monetary_value_cal,frequency_holdout,monetary_value_holdout,duration_holdout
CustomerID,,,,,,,
12346,0.0,0.0,235.0,0.0000,0.0,0.000000,90.0
12347,4.0,238.0,277.0,519.7675,2.0,26.192069,90.0
12348,2.0,110.0,268.0,297.2200,1.0,103.333333,90.0
12350,0.0,0.0,220.0,0.0000,0.0,0.000000,90.0
12352,3.0,34.0,206.0,421.7700,3.0,20.090000,90.0
...,...,...,...,...,...,...,...
18280,0.0,0.0,187.0,0.0000,0.0,0.000000,90.0
18281,0.0,0.0,90.0,0.0000,0.0,0.000000,90.0
18282,0.0,0.0,36.0,0.0000,1.0,15.568000,90.0


In [175]:
# 최종 데이터 만들기
# 고객별 RFMT가 들어간 데이터에서 frequency가 1 이상인 데이터만 가져오기!

# 아래 코드에서 보면 frequency가 0인 것은 제외하고 가져오는데, 
# 이 고객들은 전체 기간 동안 구매 일수가 1일인 유저입니다. (frequency는 총 구매일수 - 1) 
# 이들은 “반복적인” 구매를 한 고객들이 아니라서 
# BG/NBD 모형 가정에서 벗어난 고객들이기 때문에 제외하고 적합해야 합니다.

whole_filtered_df = metrics_df[metrics_df.frequency > 0]
filtered_df = metrics_cal_df[metrics_cal_df.frequency_cal > 0]

# whole_filtered_df는 L2 페널티를 최적화한 후에
# 제일 마지막에 LTV를 계산할 때 쓸 데이터이고 (calibration / holdout을 나누지 않은 데이터)
# filtered_df는 L2 페널티를 최적화하기 위해 calibration / holdout을 나눈 데이터입니다.

In [179]:
# L2 페널티 최적화하기

# hyperopt 모듈의 fmin 함수

# score_model: 실제값과 예측값의 차이에 대한 지표 (MSE / RMSE/ MAE)를 계산하는 함수
# evaluate_bgnbd_model: calibration data와 l2_reg를 넣어 BG/NBD 모형을 적합시키고, holdout data의 구매 일수 (frequency)에 대한 실제값과 예측값에 대한 MSE를 계산하는 함수
# evaluate_gg_model: calibration data와 l2_reg를 넣어 Gamma-Gamma 모형을 적합시키고, holdout data의 평균 구매 금액 (monetary value)에 대한 실제값과 예측값에 대한 MSE를 계산하는 함수


def score_model(actuals, predicted, metric = 'mse'): # score 평가

    metric = metric.lower() # 입력값을 소문자로 받음
    
    # MSE/RMSE
    if metric == 'mse' or metric == 'rmse':
        val = np.sum(np.square(actuals - predicted))/actuals.shape[0]
    elif metric == 'rmse':
        val = np.sqrt(val)

    # MAE
    elif metric == 'mae':
        val = np.sum(np.abs(actuals-predicted))/actuals.shape[0]
    else:
        val = None

    return val


def evaluate_bgnbd_model(param): # bgnbd모델

    data = inputs 
    l2_reg = param


    # 모형 적합
    model = BetaGeoFitter(penalizer_coef=l2_reg) # l2규제 적용, 과대적합 방지
    model.fit(data['frequency_cal'], data['recency_cal'], data['T_cal'])
    # fit 시 빈도, 최근, T전체기간 을 입력.
    

    # 모형 평가
    frequency_actual = data['frequency_holdout'] # 실제값
    frequency_predicted = model.predict(data['duration_holdout'], # 예측값
                                        data['frequency_cal'],
                                        data['recency_cal'],
                                        data['T_cal'])

    mse = score_model(frequency_actual, frequency_predicted, 'mse')

    return {'loss' : mse, 'status' : STATUS_OK}


def evaluate_gg_model(param): # gg모델

    data = inputs 
    l2_reg = param

    #GammaGamma 모형 적합
    model = GammaGammaFitter(penalizer_coef=l2_reg)
    model.fit(data['frequency_cal'], data['monetary_value_cal'])

    # 모형 평가
    monetary_actual = data['monetary_value_holdout'] # 실제값
    monetary_predicted = model.conditional_expected_average_profit(
                                            data['frequency_holdout'],
                                            data['monetary_value_holdout']
                                            )
    mse = score_model(monetary_actual, monetary_predicted, 'mse')

    # return score and status
    return {'loss' : mse, 'status' : STATUS_OK }



In [177]:
# BG/NBD

# beta-geometric 베타분포 기하분포의 결합
# negative-binomial-distributiond 음이항함수


# 이 후 hyperopt 모듈의 fmin 함수를 이용해 
# BG/NBD 모형과 Gamma-Gamma 모형 각각의 최적의 L2 penalty를 찾습니다.



search_space = hp.uniform('l2',0.0,1.0) #????
algo  = tpe.suggest # ??
trials = Trials() # ??
inputs = filtered_df  # l2페널티 최적화를 위해 훈련/테스트로 분리한 데이터셋

argmin = fmin(
    fn = evaluate_bgnbd_model, # 목적함수
    space = search_space,
    algo = algo,
    max_evals = 100,
    trials = trials
)

l2_bgnbd = space_eval(search_space, argmin) # ?
print(l2_bgnbd)


# 0.9998408311469851

100%|██████████| 100/100 [00:05<00:00, 18.75trial/s, best loss: 2.9943923609709566]
0.9998408311469851


In [180]:
trials = Trials()

#GammaGamma

argmin = fmin(
    fn = evaluate_gg_model,
    space = search_space,
    algo = algo,
    max_evals = 100,
    trials = trials
)

l2_gg = space_eval(search_space, argmin)
print(l2_gg)

# 0.007279394874148898

100%|██████████| 100/100 [00:01<00:00, 52.57trial/s, best loss: 428.5688405144764]
0.007279394874148898


: 

In [ ]:
# BG/NBD 모형 적합

# BetaGeoFitter (penalizer_coef) 및 predict 함수


lifetimes_model = BetaGeoFitter(penalizer_coef =l2_bgnbd) # BG/NBD l2규제 #l2_bgnbd = hyperopt로 나온 결과

# calibration 데이터의 R, F, T로 모형 적합
lifetimes_model.fit(filtered_df['frequency_cal'], filtered_df['recency_cal'], filtered_df['T_cal'])

# holdout 데이터로 모델 평가 : F의 실제 값과 예측값의 MSE
frequency_actual = filtered_Df['frequency_holdout']
frequen


In [170]:


current_date = df['InvoiceDate'].max() #2011-12-09
# cols = ['CustomerID','InvoiceNo','InvoiceDate','Sales']



# 실제로 만드는 방법
rfm_df = df.groupby('CustomerID').agg({
            'InvoiceNo': lambda x : x.nunique() -1, # frequent 첫 거래는 제거
            'InvoiceDate':[
                            lambda x  : (x.max() - x.min()).days,  # recent                          
                            lambda x : (current_date - x.min()).days # T
                            ], 
            'Sales': lambda x : x.mean()
                                            })

# data_12347.groupby('InvoiceDate').sum()[1:].mean() # 날짜별로 그룹화 한 후 구매금액의 합을 날짜별 평균을 구함

rfm_df.columns = ['frequent','recent','T','monetary']

print(rfm_df.head())

# R : 고객별 첫 구매 ~ 마지막 구매까지의 시간
# F : 고객별 구매 일 수 (첫구매는 포함하지 않는다.)
# M : 고객별 평균 구매금액(날짜별)
# T : 고객별 첫 구매 ~ 집계일까지의 시간

            frequent  recent    T      monetary
CustomerID                                     
12346              0       0  325  77183.600000
12347              6     365  367     23.681319
12348              3     283  358     57.975484
12349              0       0   18     24.076027
12350              0       0  310     19.670588


# Data Preprocessing

# RFM Analysis

RFM은 Recency, Frequency 및 Money value를 나타내며, 각각은 몇 가지 주요 고객 특성에 해당합니다. 이러한 RFM 측정 기준은 빈도와 금전적 가치가 고객의 평생 가치에 영향을 미치고, 근래에는 참여의 척도인 유지에 영향을 미치기 때문에 고객의 행동을 나타내는 중요한 지표입니다.

RFM 요인은 다음과 같은 사실을 보여줍니다.

최근 구매일수록 고객이 판촉 활동에 더 많은 반응을 보입니다.
고객이 더 자주 구매할수록 참여도와 만족도가 높아집니다.
금전적 가치는 많은 소비자와 낮은 가치의 구매자를 구별합니다.

최근 자주 돈을 많이 쓴 고객


In [ ]:
print(df['InvoiceDate'].min())
print(df['InvoiceDate'].max())

In [ ]:
today_date = dt.datetime(2011,12,11)
# print(today_date) #2011-12-11 00:00:00 오늘이라고 가정, 데이터의 마지막날 하루 뒤

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate' : lambda x : (today_date - x.max()).days,
    'InvoiceNo' : lambda x : x.nunique(),
    'TotalPrice' : lambda x : x.sum()
})

# 고객 아이디로 그룹화
# 송장 일자를 오늘 날자에서 최대 날짜(마지막 구매날짜)를 뺀 날짜로, 3이면 최근으로부터 3일전에 구매함을 의미 (최근의 정의)
# 송장 번호를 유니크한 개수로, 5 이면 해당 기간동안 5개의 별도의 주문을 함을 의미 (빈도의 정의)
# 구매 금액을 다 더함 (금전가치의 정의)

rfm.columns = ['recency','frequency','monetary']
# rfm['monetary'] = rfm[rfm['monetary'] > 0]

rfm = rfm.reset_index()

print(rfm)


In [ ]:
print(rfm['monetary'].sort_values(ascending =False))

# RFM Scores



In [ ]:
def get_rfm_scores(dataframe):
    
    df_ = dataframe.copy()

    #qcut 은 데이터프레임 내의 데이터를 동일한 숫자로 등분함 # 5등분 
    # 5점 ~ 1점으로 스코어제, labels에 따라 작은값->큰값으로 0번째 레이블, 1번째.. 로 매김
    # 최근 점수
    df_['recency_score'] = pd.qcut(df_['recency'],5,labels=[5,4,3,2,1])

    # df['a'].rank()함수는  method=first는 동점 관측치 중 먼저 나타나는 값을 높은 순위를 줌
    # 빈도 점수
    df_['frequency_score'] = pd.qcut(df_['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
    # 금전 가치 점수
    df_['menetary_score'] = pd.qcut(df_['monetary'],5,labels=[1,2,3,4,5])
    
    df_['RFM_SCORE'] = (df_['recency_score'].astype(str)) + (df_['frequency_score'].astype(str))

    return df_

rfm = get_rfm_scores(rfm)



In [ ]:
rfm


# Segmentation

10개의 등급으로 나누었으면 이는 각 기업의 기준마다 다르다.

RFM 3가지 기준에 1~5점의 각각 점수를 부여함
recency, frequency, monetary 순
최근과 자주 구매하는지만 고려

1. 최근에 구매하지 않았고 자주 구매하지 않음 : hibernating(휴면)
2. 최근에 구매하지 않았으나 구매시 종종 구매함 : at_Risk(위험)
3. 최근에는 구매하지 않았으나 구매 빈도가 아주 높음 : cant_loose(놓쳐서 안되는 고객)
4. 2달전 쯤 구매했지만 자주 구매하지 않음 : about_to_sleep(이탈 우려 고객)
5. 2달전 쯤 구매했지만 여태 어느정도 자주 구매함 : need_attention(집중 해야할 고객)
6. 2달전 쯤 구매하고 자주 물건을 구매함 : loyal_customers(충성 고객)
7. 꽤 최근 구매, 처음 구매 : promising(최근 유입 고객)
8. 최근 구매, 처음 구매 : new_customers(신규유저)
9. 꽤 최근에 구매하고 빈도도 적당함 : potential_loyalists(잠재 우량 고객)
10. 최근 구매 다수 구매 : champions(최고 고객)

        r'[1-2][1-2]': 'hibernating',
           r'[1-2][3-4]': 'at_Risk',
           r'[1-2]5': 'cant_loose',
           r'3[1-2]': 'about_to_sleep',
           r'33': 'need_attention',
           r'[3-4][4-5]': 'loyal_customers',
           r'41': 'promising',
           r'51': 'new_customers',
           r'[4-5][2-3]': 'potential_loyalists',
           r'5[4-5]': 'champions'}



In [ ]:
seg_map = {r'[1-2][1-2]': 'hibernating',
           r'[1-2][3-4]': 'at_Risk',
           r'[1-2]5': 'cant_loose',
           r'3[1-2]': 'about_to_sleep',
           r'33': 'need_attention',
           r'[3-4][4-5]': 'loyal_customers',
           r'41': 'promising',
           r'51': 'new_customers',
           r'[4-5][2-3]': 'potential_loyalists',
           r'5[4-5]': 'champions'}


rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

rfm

# Segmentation Map

In [ ]:
segments = rfm['segment'].value_counts().sort_values(ascending = False)
segments.iplot(kind='bar')

In [ ]:
# treemap 차트로 표현

segments = rfm['segment'].value_counts().sort_values(ascending = False)
# 밸류의 숫자를 센 세그먼트 컬럼을 변수에 저장 champions = 3090, new_customer= 500 등

from plotly import express as px

fig = px.treemap(
    segments, # 데이터셋
    names = list(segments.index),
    path= ['segment'], # 데이터 프레임 내 어떤 컬럼을 사용할 것인지
    values= segments.values, # 데이터의 값을 비중으로 나타냄
    title= 'segments비중' # 차트의 제목을 나타냄
)

fig.show()

In [ ]:
# segments = rfm['segment'].value_counts().sort_values(ascending = False)
# fig = plt.gcf()
# ax = fig.add_subplot()
# fig.set_size_inches(16, 10)
# squarify.plot(sizes=segments,
#               label=[label for label in seg_map.values()],
#               color=['#AFB6B5', '#F0819A', '#926717', '#F0F081', '#81D5F0',
#                      '#C78BE5', '#748E80', '#FAAF3A', '#7B8FE4', '#86E8C0'],
#               pad = False,
#               bar_kwargs = {'alpha': 1},
#               text_kwargs = {'fontsize':15})
# plt.title("Customer Segmentation Map", fontsize = 20)
# plt.xlabel('Frequency', fontsize = 18)
# plt.ylabel('Recency', fontsize = 18)
# plt.show()

# Model Evaluation

In [ ]:
from sklearn.metrics import (silhouette_score,
                             calinski_harabasz_score,
                             davies_bouldin_score)

X = rfm[['recency_score','frequency_score']]
# print(X)

labels = rfm['segment']
# print(labels)

print('관측 개수 : {}'.format(X.shape[0]))
print('세그먼트 개수 : {}'.format(labels.nunique()))
print('실루엣 계수 : {}'.format(round(silhouette_score(X,labels),3)))

# 실루엣 계수(Silhouette Coefficient) : 

# 각 데이터 포인트와 주위 데이터 포인트들과의 거리 계산을 통해 값을 구하며,
# 군집 안에 있는 데이터들은 잘 모여있는지,
# 군집끼리는 서로 잘 구분되는지 클러스터링을 평가하는 척도로 활용된다.

# * 참고한 논문의 표현을 빌리자면, 군집 내 비유사성('within' dissimilarities)은 작고, 
# 군집 간 비유사성('between' dissimilarities)은 커야 생성된 클러스터의 품질이 좋다고 할 수 있다. 



print('칼린스키-하라바즈 스코어 : {}'.format(round(calinski_harabasz_score(X, labels),3)))
print('다비스 볼딘 스코어 : {}'.format(round(davies_bouldin_score(X, labels),3)))
print(''.format())

# Segment Analysis


In [ ]:
rfm[['recency','monetary','frequency','segment']].groupby('segment').agg({'mean','std','max','min'})

In [ ]:
plt.figure(figsize = (18, 8))
ax = sns.countplot(data = rfm,
                   x = 'segment',
                   palette = palette)
total = len(rfm.segment)
for patch in ax.patches:
    percentage = '{:.1f}%'.format(100 * patch.get_height()/total)
    x = patch.get_x() + patch.get_width() / 2 - 0.17
    y = patch.get_y() + patch.get_height() * 1.005
    ax.annotate(percentage, (x, y), size = 14)
plt.title('Number of Customers by Segments', size = 16)
plt.xlabel('Segment', size = 14)
plt.ylabel('Count', size = 14)
plt.xticks(size = 10)
plt.yticks(size = 10)
plt.show()

In [ ]:
plt.figure(figsize = (18, 8))
sns.scatterplot(data = rfm, x = 'recency',
                y = 'frequency', hue ='segment', palette= palette, s = 60)
plt.title('Recency & Frequency by Segments', size = 16)
plt.xlabel('Recency', size = 12)
plt.ylabel('Frequency', size = 12)
plt.xticks(size = 10)
plt.yticks(size = 10)
plt.legend(loc = 'best', fontsize = 12,
           title = 'Segments',title_fontsize=14)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize = (18, 8))
fig.suptitle('RFM Segment Analysis', size = 14)
feature_list = ['recency', 'monetary', 'frequency']
for idx, col in enumerate(feature_list):
    sns.boxplot(ax = axes[idx], data = rfm,
                x = 'segment', y = feature_list[idx],
                palette= palette)
    axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=60)
    if idx == 1:
        axes[idx].set_ylim([0, 400])
    if idx == 2:
        axes[idx].set_ylim([0, 30])
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(16, 12))
fig.suptitle('RFM Segment Analysis', size = 14)
feature_list = ['recency', 'monetary', 'frequency']
for idx, col in enumerate(feature_list):
    sns.histplot(ax = axes[idx], data = rfm,
                 hue = 'segment', x = feature_list[idx],
                 palette= palette)
    if idx == 1:
        axes[idx].set_xlim([0, 400])
    if idx == 2:
        axes[idx].set_xlim([0, 30])
plt.tight_layout()
plt.show()


# Cohort Analysis

코호트는 앱 가입 날짜, 첫 구매 월, 지리적 위치, 획득 채널(유기 사용자, 공연 마케팅 등) 등 공통점을 공유하는 사람들의 모임이다. 코호트 분석에서, 우리는 몇 가지 일반적인 패턴이나 행동을 식별하기 위해 이러한 사용자 그룹을 시간에 따라 추적한다.

In [ ]:
def cohortAnalysis(dataframe):
    # 1. 데이터프레임을 카피합니다.
    data = dataframe.copy() 
    # 2. 고객아이디, 송장번호, 송장일자에서 중복을 제외한 데이터를 저장합니다.
    data = data[['CustomerID','InvoiceNo','InvoiceDate']].drop_duplicates() 
    # 3. 주문 월이라는 새 컬럼을 생성하고 송장 날짜에서 월만 추출한 데이터를 저장합니다.
    data['order_month'] = data['InvoiceDate'].dt.to_period('M')
    # 4. 코호트 컬럼을 만듭니다. 고객아이디로 그룹화 한 후 송장일자컬럼의 최저(처음 구매날짜)의 월을 추출
    data['cohort'] = data.groupby('CustomerID')['InvoiceDate'].transform('min').dt.to_period('M')
    # 5. 고객 아이디의 유니크 값을 세그먼트로 분류한 그룹에서 뽑아냄
    cohort_data = data.groupby(['cohort','order_month']).agg(n_customers=('CustomerID','nunique')).reset_index(drop=False)
    # cohort_data = data.groupby('segment')['CustomerID'].count()

    # 6. 
    cohort_data['period_number'] = (cohort_data.order_month - cohort_data.cohort).apply(attrgetter('n'))

    # 7.
    cohort_pivot = cohort_data.pivot_table(
        index = 'cohort',
        columns = 'period_number',
        values = 'n_customers'
    )

    # 8. 
    cohort_size = cohort_pivot.iloc[:,0]

    # 9.
    retention_matrix = cohort_pivot.divide(cohort_size, axis=0)

    # 10.
    with sns.axes_style('white'):
        fig, ax = plt.subplots(1, 2, figsize=(12,8),
        sharey=True,
        gridspec_kw ={ 'width_ratios':[1,11]})

        sns.heatmap(
            retention_matrix,
            mask = retention_matrix.isnull(),
            annot = True,
            cbar = False,
            fmt = '.0%',
            cmap = 'coolwarm',ax= ax[1]
        )

        ax[1].set_title('월별 코호트 : 유저 리텐션', fontsize=14)

        ax[1].set(
            xlabel='기간',
            ylabel=' '
        )

        white_cmap = mcolors.ListedColormap(['white'])
        
        sns.heatmap(pd.DataFrame(cohort_size).rename(columns={0 : 'cohort_size'}),
            annot = True,
            cbar = False,
            fmt = '0%',
            cmap = 'coolwarm', ax = ax[1]
        )
        
        ax[1].set_title('Monthly Cohort: User Retention', fontsize=14)

        ax[1].set(xlabel='# of periods', ylabel=' ')

        white_cmap = mcolors.ListedColormap(['white'])

        sns.heatmap(pd.DataFrame(cohort_size).rename(columns={0:'cohort_size'}),
            annot=True,
            cbar=False,
            fmt='g',
            cmap=white_cmap,
            ax=ax[0]
        )
        
        fig.tight_layout()

cohortAnalysis(df)


    



In [ ]:
def CohortAnalysis(dataframe):
    
    data = dataframe.copy()
    data = data[['CustomerID', 'InvoiceNo', 'InvoiceDate']].drop_duplicates()
    data['order_month'] = data['InvoiceDate'].dt.to_period('M')
    data['cohort'] = data.groupby('CustomerID')['InvoiceDate']\
    .transform('min').dt.to_period('M')
    cohort_data = data.groupby(['cohort', 'order_month'])\
    .agg(n_customers=('CustomerID', 'nunique')).reset_index(drop=False)
    cohort_data['period_number'] = (cohort_data.order_month - cohort_data.cohort)\
    .apply(attrgetter('n'))
    cohort_pivot = cohort_data.pivot_table(index = 'cohort',
                                           columns = 'period_number',
                                           values = 'n_customers')
    cohort_size = cohort_pivot.iloc[:,0]
    retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)
    with sns.axes_style("white"):
        fig, ax = plt.subplots(1, 2, figsize=(12, 8),
                        sharey=True,
                        gridspec_kw={'width_ratios': [1, 11]})
        sns.heatmap(retention_matrix, 
                    mask = retention_matrix.isnull(), 
                    annot = True,
                    cbar = False,
                    fmt='.0%', 
                    cmap='coolwarm', ax=ax[1])
        ax[1].set_title('Monthly Cohorts: User Retention', fontsize=14)
        ax[1].set(xlabel='# of periods',
                  ylabel='')
        white_cmap = mcolors.ListedColormap(['white'])
        sns.heatmap(pd.DataFrame(cohort_size).rename(columns={0: 'cohort_size'}), 
                    annot=True, 
                    cbar = False,
                    fmt='g',
                    cmap=white_cmap,
                    ax=ax[0])
        fig.tight_layout()
    
CohortAnalysis(df)



# Customer Lifetime Value
고객 평생 가치는 고객이 지불 고객으로서 전체 기간 동안 얼마나 많은 돈을 브랜드에 가져다 줄 것인가?

https://assaeunji.github.io/statistics/2022-04-15-ltv/

LTV는 확률 기반 모형으로 특정한 시점 t에서 고객마다 어느정도의 생애가치를 가져다 주는지 측정

고객의 과거 구매정보를 가지고 미래의 예상 구매횟수 * 예상 평균수익으로 계산
----

고객의 과거 구매정보 :  
    1. 첫 구매 ~ 집계일까지의 시간(T)
    2. 첫 구매 ~ 마지막 구매일자까지의 시간(Recency ; tx - t0)
    3. 첫 구매 ~ 집계일까지의 구매 횟수(Frequency ; x)
    4. 구매 건 마다의 구매 금액(Monetary Value ; mx)

미래의 예상 구매횟수 RFM분석을 통한 데이터를 바탕으로 BG/NBD 모델로 추정

미래의 예상 구매금액은 Monetary Value의 정보를 가지고 Gamma-Gamma모델로 추정

앞에서 확률적으로 모델을 추정한다 말씀을 드렸는데요. 모델 이름도 사실 확률 분포의 이름에서 따왔습니다. 
BG/NBD는 베타 분포와 기하 분포의 결합과, 음이항 분포로 구성되고(Beta-Geometric / Negative Binoimal Distribution), 
Gamma-Gamma는 두 감마 분포의 결합으로 이루어집니다.



BTYD 모델

구체적인 모형을 설명하기에 앞서 “죽을 때까지 구매하는” (BTYD; Buy Till You Die) 모형어감이 그러니 이제부터 BTYD 모형이라 부르겠습니다의 컨셉부터 설명드리겠습니다. BTYD 모형을 바탕으로 BG/NBD 모형을 구성합니다.

BTYD 모델은 과거의 구매 횟수와 구매 금액과 같은 데이터를 바탕으로 고객의 생애 가치를 계산합니다. BTYD는 다음과 같은 질문에 답할 수 있습니다.

현재 얼마나 많은 고객들이 남아있나요?
지금으로부터 1년 뒤에 얼마나 많은 고객들이 남아있을까요?
어떤 고객들이 이탈했었나요?
고객들이 한 회사에 미래에 얼마를 소비할까요?

---------------------------------------------------------------

1. 구매 (Transaction) 프로세스 (Buy)

고객이 남아있는 동안, 고객의 구매 횟수는 일정한 기간 동안의 구매율을 모수로 하는 포아송 분포를 따릅니다. 사실 포아송 분포는 “구매 횟수”에 대한 분라 구매할 확률을 모수로 하지 않고, 정해진 시간 동안의 예상 구매 횟수를 모수로 하는데요.

여기서 전 왜 원 논문에서는 모수를 구매 횟수가 아닌 구매율 (transaction rate)로 표현하는가가 이해가 되지 않았었죠. 아마도 일정한 기간을 어떻게 정의하느냐에 따라서 충분히 비율로 표현할 수 있기 때문이라 생각이 듭니다. 예를 들어, 1년에 30번 구매한 고객이 있다면 구매 횟수가 Pois (30)을 따른다고 말할 수도 있겠지만, 단위 기간을 이보다 더 좁은 기간인 1일로 정의한다면 1일에 (30 / 12개월) / 30일 = 1/12의 확률로 구매를 한다 말할 수 있기 때문에 Pois(1/12)라고 말할 수 있는 것이죠.

이 구매율은 고객마다 다르다고 가정하고, 전체 고객들의 구매율은 Gamma(r,α)를 따른다 가정합니다.

2. 이탈 (Dropout) 프로세스 (Till You Die)

각 고객들은 이탈률 p를 가집니다. 즉, 고객들은 구매를 한 후에 특정한 확률로 이탈함을 가정하는 것입니다. 이러한 이탈 확률은 고객마다 달라 Beta (a,b)를 따른다 가정합니다. 보통 확률에 대한 분포를 가정할 때 범위가 (0,1)로 정해져있는 베타 분포를 가정하곤 하는데, 감마 분포를 가정한 이유도 위와 같은 이유이지 않을까 생각해봅니다.


위에서 말했듯 BG/NBD 모형은 BTYD 모형에 기반하여 분포를 가정합니다. 구체적으로 분포 가정은 다음과 같습니다.

1. 고객이 남아있는 동안, 일정한 기간 T 동안의 구매 횟수는 Pois(λT)를 따릅니다. 위에서 말씀 드렸듯이 1일 간 Pois (1/12)를 따른다면 T= 1년일 경우 Pois (30)을 따르게 되겠죠! (포아송의 모수인 λT는 예상 구매 횟수 를 의미합니다)

2. 고객마다 일정한 기간 동안 구매하는 횟수는 다릅니다. 이는 λ~ Gamma (r, α)을 따릅니다.

3. j번째 구매가 마지막이고 더 이상 구매를 하지 않을 확률 (이탈률)은 p입니다. 이탈할 때까지의 구매 횟수는 Geo (p)를 따릅니다.

4. 고객마다 더 이상 구매를 하지 않을 확률 (이탈률)은 다릅니다. 이탈률 p는 p ~ Beta (a,b)를 따릅니다.

5. 고객별 일정 기간 동안의 구매 횟수와 구매를 하지 않을 확률은 서로 영향을 주지 않습니다.



모형 이름이 BG/NBD인데 

B에 해당하는 베타 분포 (Beta Distribution), 
G에 해당하는 기하 분포 (Geometric Distribution)

NBD에 해당하는 음이항 분포 (Negative Binomial Distribution) 

포아송 분포와 감마 분포의 결합이 바로 음이항 분포 (Negative Binomial Distribution)

이를 Poisson-Gamma Mixture라 부릅니다.

포아송 분포: 단위 시간 동안의 성공 횟수에 대한 분포
감마 분포: 사건을 n번 시행할 때까지의 총 시간에 대한 분포
기하 분포: 사건이 1번 발생할 때까지의 시도 횟수에 대한 분포
베타 분포: 확률에 대한 분포 (범위: (0,1))


날짜	구매 금액
2022.01.16	44,159원
2022.01.24	44,385원
2022.02.03	40,700원
2022.02.05	43,520원
2022.02.26	48,140원
2022.03.16	27,186원
2022.03.23	37,161원
2022.04.10	40,060원



포아송 분포: 네 달에 거쳐 8번 구매했으니 한 달에 2번 구매한 꼴이 됩니다. 이처럼 한 달의 평균 구매 횟수를 Pois (λ)를 따른다 가정합니다.

감마 분포: 위 내용을 바꿔 말하면 두 번 구매할 때까지 한 달이라는 시간이 걸렸다고 말할 수 있습니다. 이처럼 r번 구매할 때까지 걸리는 시간은 Gamma(r,α)를 따른다 가정합니다.

기하 분포: 여기서의 사건은 “이탈”입니다. 이탈할 때까지의 총 구매 횟수는 Geo(p)를 따른다 가정합니다.

베타 분포: 마지막으로 이 이탈률 p는 Beta(a,b)를 따른다 가정합니다.



BG/NBD 모델은 고객마다 다른 구매 횟수를 모델링했다면, Gamma-Gamma 모델은 고객별 구매 금액 정보들을 이용합니다.


Gamma-Gamma 모델은 다음과 같은 가정을 하게 됩니다.

1. 고객별 구매 금액은 평균 구매 금액을 중심으로 랜덤하게 분포합니다.
2. 고객들의 평균 구매 금액은 고객마다 다르지만, 한 고객의 평균 구매 금액은 시간에 따라 불변합니다.
3. 이 평균 구매 금액은 구매 과정과 독립이여야 합니다.


이에 따라 다음과 같이 분포를 가정하여 가능도를 구하게 됩니다.

1. 각 구매 건별 구매 금액 zi z i 는 zi∼ z i ∼  Gamma (p,v)를 따른다 가정합니다. 
총 x번 구매하면 ∑xi=1zi∼ ∑ i = 1 x z i ∼  Gamma (px,v)를 따르고, 평균 구매 금액인 mx=∑xi=1zi/x m x = ∑ i = 1 x z i / x 는 ∼ ∼  Gamma(px, vx)을 따르게 됩니다. 
(Gamma 분포를 따르는 확률 변수의 합도 Gamma분포를 따릅니다. 자세한 내용은 위키 참조!) 

2. 고객마다 평균 구매 금액이 다르기 때문에 scale 모수인 v 또한 v∼ v ∼ Gamma (q,γ γ )를 따른다 가정합니다. 

3. 이 둘의 관계를 갖고 베이즈 정리를 활용한다면 평균 구매 금액의 marginal 분포는 Gamma (px+q, γ+mxx γ + m x x )를 따르게 됩니다. (mx=∑xi=1zi/x m x = ∑ i = 1 x z i / x  평균 구매 금액)



고객마다 학습에 들어가는 T (Time), R (Recency), F (Frequency),M (Monetary) 정보와 결합하여

구매 빈도 (F)는 낮은데 LTV가 높은 고객
구매 빈도 (F), 평균 구매 금액 (M)이 높고 LTV가 높은 고객
최근성 (R)은 떨어지는데, LTV가 높은 고객



In [ ]:
# Customer Lifetime Value


cltv_df = df.groupby('CustomerID').agg({
    'InvoiceDate': [lambda x : (x.max() - x.min()).days,
                    lambda x : (today_date - x.min()).days], # 이런식으로 리스트 안에 두가지 식을 적용 시 컬럼이 각각 만들어짐
    'InvoiceNo': lambda x : x.nunique(),
    'TotalPrice' : lambda x : x.sum()
})

# cltv_df.head()
# InvoiceDate	InvoiceNo	TotalPrice
# <lambda_0>	<lambda_1>	<lambda>	<lambda>

# T 컬럼은 오늘부터 가장 처음 구매한 기간의 차로 구매 기간을 의미
cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
cltv_df.head()

In [ ]:
# Average Order Value
cltv_df['monetary'] = cltv_df['monetary'] / cltv_df['frequency']

# Recency & Tenure
cltv_df['recency'] = cltv_df['recency'] / 7
cltv_df['T'] = cltv_df['T'] / 7

# Frequency
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

cltv_df

# BG/NBD


베타 기하학적 / 음의 이항 분포 모델은 각 고객이 수행한 트랜잭션 수가 감마 분포에 이어 고객 간 트랜잭션 속도의 이질성을 갖는 포아송 프로세스를 따른다고 가정합니다. 

이러한 가정은 고객이 살아있는 동안 수행한 트랜잭션 수를 모델링하기 위한 음의 이항 분포(NBD)를 제공합니다. 
우리는 BetaGeoFitter와 Life times 패키지를 사용하여 BG/NBD 모델을 구축할 수 있습니다.

In [ ]:
BGF = BetaGeoFitter(penalizer_coef= 0.001) #과대적합 방지

BGF.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

        

In [ ]:
BGF.conditional_expected_number_of_purchases_up_to_time(1,
                                                        cltv_df['frequency'],
                                                        cltv_df['recency'],
                                                        cltv_df['T']
                                                       ).sort_values(ascending = False)\
.head(10)\
.to_frame('Expected Number of Transactions')\
.reset_index()

In [ ]:
BGF.conditional_expected_number_of_purchases_up_to_time(4,
                                                        cltv_df['frequency'],
                                                        cltv_df['recency'],
                                                        cltv_df['T']
                                                       ).sort_values(ascending = False)\
.head(10)\
.to_frame('Expected Number of Transactions')\
.reset_index()

In [ ]:
#Plot a figure with period actual and predicted transactions.
plot_period_transactions(BGF,
                         max_frequency = 7)
plt.show()